In [ ]:
import numpy as np
import os
import pandas as pd

np.random.seed(0)

In [ ]:
def load_file(filepath):
	dataframe = pd.read_csv(filepath, header=None, delim_whitespace=True)
	return dataframe.values

In [ ]:
test = load_file('dataset/UCI/train/X_train.txt')

In [ ]:
# load a single file as a numpy array
def load_file(filepath):
	dataframe = pd.read_csv(filepath, header=None, delim_whitespace=True)
	return dataframe.values
 
# load a list of files, such as x, y, z data for a given variable
def load_group(filenames, prefix=''):
	loaded = list()
	for name in filenames:
		data = load_file(prefix + name)
		loaded.append(data)
	# stack group so that features are the 3rd dimension
	loaded = np.dstack(loaded)
	return loaded
 
# load a dataset group, such as train or test
def load_dataset(group, prefix=''):
	filepath = prefix + group + '/inertia/'
	filenames = list()
	# body acceleration
	filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
	# body gyroscope
	filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
	# load input data
	X = load_group(filenames, filepath)
	# load class output
	y = load_file(prefix + group + '/y_'+group+'.txt')
	return X, y

def segmentData(accData,time_step,step):
    segmentAccData = list()
    for i in range(0, accData.shape[0] - time_step,step):
        segmentAccData.append(accData[i:i+time_step,:])
    return segmentAccData
    
# load all train
trainX, trainy = load_dataset('train', 'dataset/UCI/')
trainy = np.asarray([x - 1 for x in trainy])

# print(trainX.shape, trainy.shape)
# load all test
testX, testy = load_dataset('test', 'dataset/UCI/')
# soo it starts at 0
testy = np.asarray([x - 1 for x in testy])

In [ ]:
datasets = list()
for x in range(0,trainX.shape[2]):
    datasets.append(np.concatenate((trainX[:,:,x],testX[:,:,x]), axis = 0))
datasets = np.dstack(datasets)

In [ ]:
def feature_normalize(dataset,mu,sigma):
    return (dataset - mu)/sigma

In [ ]:


meanAcc = np.mean(datasets[:,:,:3])
stdAcc = np.std(datasets[:,:,:3])
varAcc = np.var(datasets[:,:,:3])
stackedFeaturesAcc = np.hstack((meanAcc,stdAcc,varAcc))

meanGyro = np.mean(datasets[:,:,3:])
stdGyro = np.std(datasets[:,:,3:])
varGyro = np.var(datasets[:,:,3:])
stackedFeaturesGyro = np.hstack((meanGyro,stdGyro,varGyro))

normalizedTrainAcc = (trainX[:,:,:3] - meanAcc) / stdAcc
normalizedTrainGyro = (trainX[:,:,3:] - meanGyro) / stdGyro

normalizedTestAcc = (testX[:,:,:3] - meanAcc) / stdAcc
normalizedTestGyro = (testX[:,:,3:] - meanGyro) / stdGyro


normalizedAllAcc = (datasets[:,:,:3] - meanAcc) / stdAcc
normalizedAllGyro = (datasets[:,:,3:] - meanGyro) / stdGyro

stackedFeatures = np.vstack((stackedFeaturesAcc,stackedFeaturesGyro))

In [ ]:
normalizedX = np.dstack((normalizedTrainAcc,normalizedTrainGyro))
normalizedEval = np.dstack((normalizedTestAcc,normalizedTestGyro))
normalizedAll = np.dstack((normalizedAllAcc,normalizedAllGyro))

In [ ]:
dataName = 'UCI'
os.makedirs('datasetStand/'+dataName+ '/train', exist_ok=True)
np.savetxt('datasetStand/'+dataName+ '/train/AccX'+dataName+'.csv', normalizedX[:,:,0], delimiter=',')
np.savetxt('datasetStand/'+dataName+ '/train/AccY'+dataName+'.csv', normalizedX[:,:,1], delimiter=',')
np.savetxt('datasetStand/'+dataName+ '/train/AccZ'+dataName+'.csv', normalizedX[:,:,2], delimiter=',')
np.savetxt('datasetStand/'+dataName+ '/train/GyroX'+dataName+'.csv', normalizedX[:,:,3], delimiter=',')
np.savetxt('datasetStand/'+dataName+ '/train/GyroY'+dataName+'.csv', normalizedX[:,:,4], delimiter=',')
np.savetxt('datasetStand/'+dataName+ '/train/GyroZ'+dataName+'.csv', normalizedX[:,:,5], delimiter=',')
np.savetxt('datasetStand/'+dataName+ '/train/Label'+dataName+'.csv', trainy, delimiter=',')

os.makedirs('datasetStand/'+dataName+ '/eval', exist_ok=True)
np.savetxt('datasetStand/'+dataName+ '/eval/AccX'+dataName+'.csv', normalizedEval[:,:,0], delimiter=',')
np.savetxt('datasetStand/'+dataName+ '/eval/AccY'+dataName+'.csv', normalizedEval[:,:,1], delimiter=',')
np.savetxt('datasetStand/'+dataName+ '/eval/AccZ'+dataName+'.csv', normalizedEval[:,:,2], delimiter=',')
np.savetxt('datasetStand/'+dataName+ '/eval/GyroX'+dataName+'.csv', normalizedEval[:,:,3], delimiter=',')
np.savetxt('datasetStand/'+dataName+ '/eval/GyroY'+dataName+'.csv', normalizedEval[:,:,4], delimiter=',')
np.savetxt('datasetStand/'+dataName+ '/eval/GyroZ'+dataName+'.csv', normalizedEval[:,:,5], delimiter=',')
np.savetxt('datasetStand/'+dataName+ '/eval/Label'+dataName+'.csv', testy, delimiter=',')

os.makedirs('datasetStand/'+dataName+ '/all', exist_ok=True)
np.savetxt('datasetStand/'+dataName+ '/all/AccX'+dataName+'.csv', normalizedAll[:,:,0], delimiter=',')
np.savetxt('datasetStand/'+dataName+ '/all/AccY'+dataName+'.csv', normalizedAll[:,:,1], delimiter=',')
np.savetxt('datasetStand/'+dataName+ '/all/AccZ'+dataName+'.csv', normalizedAll[:,:,2], delimiter=',')
np.savetxt('datasetStand/'+dataName+ '/all/GyroX'+dataName+'.csv', normalizedAll[:,:,3], delimiter=',')
np.savetxt('datasetStand/'+dataName+ '/all/GyroY'+dataName+'.csv', normalizedAll[:,:,4], delimiter=',')
np.savetxt('datasetStand/'+dataName+ '/all/GyroZ'+dataName+'.csv', normalizedAll[:,:,5], delimiter=',')
np.savetxt('datasetStand/'+dataName+ '/all/Label'+dataName+'.csv', testy, delimiter=',')

os.makedirs('datasetStand/'+dataName+ '/features', exist_ok=True)
np.savetxt('datasetStand/'+dataName+ '/features/mean-std-var'+dataName+'.csv', stackedFeatures, delimiter=',')